# Importing Modules

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
pd.plotting.register_matplotlib_converters()
import numpy as np
%matplotlib inline
import seaborn as sns
import sqlite3
print("Imported")

Imported


In [2]:
conn = sqlite3.connect('crimes.db')
cursor = conn.cursor()
print('Connected')

Connected


In [3]:
url = "https://data.sfgov.org/api/views/wg3w-h783/rows.csv?accessType=DOWNLOAD"
crimes = pd.read_csv(url)
print("Loaded")

C:\Users\amara\AppData\Local\Temp\ipykernel_30904\3489585925.py:2: DtypeWarning: Columns (18,21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  crimes = pd.read_csv(url)


Loaded


# Cleaning the Data

## Rewriting Column Names

In [4]:
crimes.columns
crimes.columns = crimes.columns.str.lower().str.replace(" ","_")
crimes.columns

Index(['row_id', 'incident_datetime', 'incident_date', 'incident_time',
       'incident_year', 'incident_day_of_week', 'report_datetime',
       'incident_id', 'incident_number', 'cad_number', 'report_type_code',
       'report_type_description', 'filed_online', 'incident_code',
       'incident_category', 'incident_subcategory', 'incident_description',
       'resolution', 'intersection', 'cnn', 'police_district',
       'analysis_neighborhood', 'supervisor_district',
       'supervisor_district_2012', 'latitude', 'longitude', 'point',
       'data_as_of', 'data_loaded_at', 'neighborhoods',
       'esncag_-_boundary_file',
       'central_market/tenderloin_boundary_polygon_-_updated',
       'civic_center_harm_reduction_project_boundary',
       'hsoc_zones_as_of_2018-06-05', 'invest_in_neighborhoods_(iin)_areas',
       'current_supervisor_districts', 'current_police_districts'],
      dtype='object')

## Parsing Dates

In [ ]:
crimes['incident_datetime'] = pd.to_datetime(crimes['incident_datetime'], errors = 'coerce')
crimes['incident_date'] = pd.to_datetime(crimes['incident_date'], errors = 'coerce')
crimes['incident_time'] = pd.to_datetime(crimes['incident_time'], errors = 'coerce')
#crimes['incident_year'] = pd.to_datetime(crimes['incident_year'], errors = 'coerce')
#crimes['Incident Day of Week'] = pd.to_datetime(crimes['Incident Day of Week'], errors = 'coerce')
crimes['report_datetime'] = pd.to_datetime(crimes['report_datetime'], errors = 'coerce')

crimes.head()

C:\Users\amara\AppData\Local\Temp\ipykernel_30904\1577822474.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crimes['incident_datetime'] = pd.to_datetime(crimes['incident_datetime'], errors = 'coerce')


## Finding and Fixing Nulls

In [ ]:
cad_null = crimes[crimes['cad_number'].isnull()]
online_null = crimes[crimes['filed_online'].isnull()]
category_null = crimes[crimes['incident_category'].isnull()]
sub_null = crimes[crimes['incident_subcategory'].isnull()]
intersection_null = crimes[crimes['intersection'].isnull()]
cnn_null = crimes[crimes['cnn'].isnull()]
hood_null = crimes[crimes['analysis_neighborhood'].isnull()]
district_null = crimes[crimes['supervisor_district'].isnull()]
district_2012_null = crimes[crimes['supervisor_district_2012'].isnull()]
lat_null = crimes[crimes['latitude'].isnull()]
long_null = crimes[crimes['longitude'].isnull()]
cord_null = crimes[crimes['point'].isnull()]
# Nulls: CAD Number, Incident Category, Incident Subcategory, Intersection (where crime happened), CNN...

In [ ]:
crimes.filed_online = crimes.filed_online.fillna(False)
print(crimes.filed_online.unique())

In [ ]:
null_percent = (crimes.isnull().sum().sum() / crimes.size) * 100
print(null_percent)

In [ ]:
crimes.dropna()
crimes.size

## Removing Duplicates

In [ ]:
crimes.drop_duplicates(subset=['row_id'])
crimes.size

# SQL Queries - Finding Trends

In [ ]:
# to sql table
crimes_sql = crimes.copy()
crimes.to_sql('crimes_sql',conn, if_exists= 'replace',index=False)

In [ ]:
# total crimes per year
total_crimes_query = '''SELECT incident_year,COUNT(*) as total_crimes FROM crimes_sql
                        WHERE report_type_code = 'II'
                        GROUP BY incident_year
                        ORDER BY incident_year ASC;'''
cursor.execute(total_crimes_query)
results = cursor.fetchall()
total_crimes = pd.DataFrame(results, columns = ['incident_year', 'total_crimes'])
print(total_crimes)

In [ ]:
# crimes by category
crimes_cat_query = '''SELECT incident_category, COUNT(*) as count
                      FROM crimes_sql
                      WHERE report_type_code = 'II'
                      GROUP BY incident_category
                      ORDER BY count DESC;
                      '''
cursor.execute(crimes_cat_query)
results = cursor.fetchall()
crime_cat = pd.DataFrame(results,columns=['incident_category','count'])
print(crime_cat.head())

In [ ]:
# crimes by district
crimes_district_query = '''SELECT police_district, COUNT(*) as count
                           FROM crimes_sql
                           WHERE report_type_code = 'II'
                           GROUP BY police_district
                           ORDER BY count DESC;
                           '''
cursor.execute(crimes_district_query)
results = cursor.fetchall()
crime_district = pd.DataFrame(results,columns=['police_district','count'])
print(crime_district.head())

In [ ]:
# crimes by neighborhood
crimes_neighborhood_query = '''SELECT analysis_neighborhood, COUNT(*) AS count
                               FROM crimes_sql
                               WHERE report_type_code = 'II'
                               GROUP BY analysis_neighborhood
                               ORDER BY count DESC;
                               '''
cursor.execute(crimes_neighborhood_query)
results = cursor.fetchall()
crime_neighborhood = pd.DataFrame(results, columns = ['analysis_neighborhood','count'])
print(crime_neighborhood.head())

In [ ]:
# arrest vs non-arrest
arrest_type = crimes.resolution.unique()
print(arrest_type)
arrest_query = '''SELECT
                     CASE
                         WHEN resolution = 'Open or Active' THEN 'Non-Arrest'
                         WHEN resolution = 'Exceptional Adult' THEN 'Arrest'
                         WHEN resolution = 'Unfounded' THEN 'Non-Arrest'
                         WHEN resolution = 'Cite or Arrest Adult' THEN 'Arrest'
                     END AS arrest_status,
                     COUNT(*) as count
                 FROM crimes_sql
                 WHERE report_type_code = 'II'
                 GROUP BY arrest_status
                 ORDER BY count DESC;'''
cursor.execute(arrest_query)
results = cursor.fetchall()
arrests = pd.DataFrame(results, columns = ['resolution','count'])
print(arrests)

In [ ]:
# crimes by week crimes
crime_day_query = '''SELECT incident_day_of_week, COUNT(*)
                     FROM crimes_sql
                     WHERE report_type_code = 'II'
                     GROUP BY incident_day_of_week
                     ORDER BY
                         CASE incident_day_of_week
                             WHEN 'Sunday' THEN 1
                             WHEN 'Monday' THEN 2
                             WHEN 'Tuesday' THEN 3
                             WHEN 'Wednesday' THEN 4
                             WHEN 'Thursday' THEN 5
                             WHEN 'Friday' THEN 6
                             WHEN 'Saturday' THEN 7
                             ELSE 8
                         END;
'''

cursor.execute(crime_day_query)
results = cursor.fetchall()
crime_day = pd.DataFrame(results,columns=['day_of_week','count'])
print(crime_day)

In [ ]:
# crime category by district
cat_by_district_query = '''SELECT police_district, incident_category, COUNT(*) as count
                           FROM crimes_sql
                           GROUP BY police_district, incident_category;'''
cursor.execute(cat_by_district_query)
results = cursor.fetchall()
cat_district = pd.DataFrame(results, columns = ['police_district','incident_category','count'])
print(cat_district)

# Seaborn Graphs and Plots

In [ ]:
# crimes over time
plt.figure(figsize=(8,4))
plt.title('Crimes Over Time')
sns.lineplot(data = total_crimes,x='incident_year',y='total_crimes')
plt.xlabel('Year')
plt.ylabel('Crimes')
plt.show()

In [ ]:
# crimes by category
plt.figure(figsize=(8,4))
plt.title("Crimes by Category")
sns.barplot(data=crime_cat.head(20),x='count',y='incident_category')
plt.xlabel("Count")
plt.ylabel("Crime Category")
plt.show()

In [ ]:
# crimes by district
plt.figure(figsize=(8,4))
plt.title('Crimes by District')
sns.barplot(data=crime_district,x='count',y='police_district')
plt.show()

In [ ]:
# crimes by district
plt.figure(figsize=(8,4))
plt.title('Crimes by Neighborhood')
sns.barplot(data=crime_neighborhood.head(20),x='count',y='analysis_neighborhood')
plt.show()

In [ ]:
total = arrests.loc[0,'count'] + arrests.loc[1,'count']
non = (arrests.loc[0,'count'] / total) * 100
arr = (arrests.loc[1,'count'] / total) * 100
arrests['percentage'] = [non,arr]
print(arrests)

plt.figure(figsize=(8,4))
plt.title("Arrests v. Non-Arrests")
colors = ['#d42619','#3232ba']
plt.pie(arrests.percentage,labels=arrests.resolution,colors=colors)
plt.show()

In [ ]:
# category - districts
top_categories = (
    cat_district.groupby('incident_category')['count']
    .sum()
    .sort_values(ascending=False)
    .head(15)
    .index
)

filtered = cat_district[cat_district['incident_category'].isin(top_categories)]

heat_data = filtered.pivot_table(
    index = 'police_district',
    columns = 'incident_category',
    values = 'count'
)

plt.figure(figsize=(16,6))
plt.title("Crime Types by District")
sns.heatmap(heat_data,annot=True,cmap='PuRd')
plt.xlabel("Crime Type")
plt.ylabel("District")
plt.show()

# Predictions and Analysis

Number of crimes (by year) decline over time with a sharp decline from 2019 to 2020
	- Crimes with null values were deleted = more null values for more recent crimes?
	- Not all 2025 crimes have been recorded yet

The most common crime category is Larceny Theft, almost 250,000
	- Followed by Malicious Mischief, Miscellaneous and Assault
	- Least: weapons carrying

The most crimes occurred in the Central District
	- Followed by Northen District and Mission District
	- Least: Park

The most crimes occurred in the Tenderloin Neighborhood
	- Followed by Mission, South of Market and Financial District/South Beach
	- Least: Pacific Heights

80% of crimes reported were non-arrest reports
	- Arrest: 593,950
	- Non-arrest: 146,796

Larceny Theft was most common in the Central District
Larceny Theft was most common in the Northen District
Recovered Vehicles were most common outside of San Francisco